<a href="https://colab.research.google.com/github/amoghantarkar/Datascience/blob/master/ANN_Churn_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.3.0'

In [ ]:
dataset=pd.read_csv("Churn_Modelling.csv")
x=dataset.iloc[:,3:-1].values
y=dataset.iloc[:,-1].values

In [ ]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])

In [ ]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
x=np.array(ct.fit_transform(x))


In [ ]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [ ]:
from sklearn.model_selection import train_test_split
# feature scaling needed for NN
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)



In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train) #only train. to avoid leakage
x_test = sc.transform(x_test)

initialize ANN, compile and train and predict


In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
# DENSE LAYER class. 
# input layer comes from input file features. 1st hidden is adding dense layer with 6 neurons
ann.add(tf.keras.layers.Dense(units=6,activation='relu')) # shallow neural network

In [ ]:
#second hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [ ]:
#final output layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid')) # for categorical - softmax

In [ ]:
#compile
# adam is shocastic gradient descent,  binary classficiation here but for categorical use categorical_crossentrophy
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']) 

In [ ]:
#hyper parameter 
ann.fit(x_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 976us/step - loss: 0.6711 - accuracy: 0.6348
Epoch 2/100
250/250 [==============================] - 0s 975us/step - loss: 0.5266 - accuracy: 0.7968
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4519 - accuracy: 0.8079
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4233 - accuracy: 0.8149
Epoch 5/100
250/250 [==============================] - 0s 919us/step - loss: 0.4111 - accuracy: 0.8196
Epoch 6/100
250/250 [==============================] - 0s 974us/step - loss: 0.4029 - accuracy: 0.8211
Epoch 7/100
250/250 [==============================] - 0s 979us/step - loss: 0.3958 - accuracy: 0.8257
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3891 - accuracy: 0.8294
Epoch 9/100
250/250 [==============================] - 0s 949us/step - loss: 0.3815 - accuracy: 0.8353
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3

# predict train dataset


In [ ]:
# need to be 2d array/matrix 
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

[[False]]


# Test Dataset

In [ ]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1506   89]
 [ 194  211]]


0.8585

prediction 
1506 correct stays
211 correct leaves
194 incorrect stays
89 incorrect leaves
